<a href="https://colab.research.google.com/github/tshihyi/real-estate-analysis/blob/main/land_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 10.3 MB/s 
     |████████████████████████████████| 358 kB 48.5 MB/s 
     |████████████████████████████████| 139 kB 56.3 MB/s 
     |████████████████████████████████| 4.1 MB 46.2 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https:/

Web Crawler

**TODO**
- [ ] if download fail, retry it

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_prefs = {"download.default_directory": './content/drive'}
chrome_options.experimental_options["prefs"] = chrome_prefs

driver = webdriver.Chrome('chromedriver', options=chrome_options)
url = "https://plvr.land.moi.gov.tw/DownloadOpenData"
driver.get(url)

try:
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'ui-id-2'))).click()
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//select[@id='historySeason_id']/option[@value='108S2']"))).click()
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//select[@id='fileFormatId']/option[@value='csv']"))).click() #csv
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@id='downloadTypeId2']"))).click() #進階
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='A_lvr_land_A']"))).click() #台北
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='F_lvr_land_A']"))).click() #新北
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='H_lvr_land_A']"))).click() #桃園
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='B_lvr_land_A']"))).click() #台中
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@value='E_lvr_land_A']"))).click() #高雄
  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,"//input[@id='downloadBtnId']"))).click()  #下載
  print("Operation successful !")
  driver.save_screenshot('shot.png')
  time.sleep(60)
except Exception:
  driver.quit()


Operation successful !


In [29]:
import pandas as pd
from zipfile import ZipFile
from pyspark.sql import SparkSession, DataFrame
from functools import reduce  # For Python 3.x

zip_file = ZipFile('./content/drive/download.zip')

df_list = [
  pd.read_csv(zip_file.open('A_lvr_land_A.csv')),
  pd.read_csv(zip_file.open('F_lvr_land_A.csv')),
  pd.read_csv(zip_file.open('H_lvr_land_A.csv')),
  pd.read_csv(zip_file.open('B_lvr_land_A.csv')),
  pd.read_csv(zip_file.open('E_lvr_land_A.csv')) 
]

Data clearning and transform

In [53]:
def covert_num(floor_name):
  result = 0
  if isinstance(floor_name, int):
    return floor_name

  if isinstance(floor_name, float):
    result = int(floor_name)
    return result

  if(floor_name.endswith('層')):
    floor_name = floor_name[:-1]
  #List of words
  num_list = {"一": 1, "二":2 , "三":3 , "四":4 , "五":5 , "六":6, "七":7 , "八":8 , "九":9 , "十":10 }
  
  for i in range (0, len(floor_name)):
    for k in num_list:
      if len(floor_name) == 1:
          if (k == floor_name[i]):
              result = num_list[k]
      elif len(floor_name) == 2:
          if (k == floor_name[0]):
            result = num_list[k] + num_list[floor_name[1]]
          else:
            result = num_list[floor_name[0]] * 10
      elif len(floor_name) == 3 :
          if (k == floor_name[1]):
            result = num_list[floor_name[0]] * 10 + num_list[floor_name[2]]
          else:
            result = 0
    return result

def convert_western_date(txn_date):
  if len(txn_date)>0:
    txn_date = txn_date.replace(txn_date[0:3],str(int(txn_date[0:3])+1911))
    txn_date = txn_date[0:4]+ "-" + txn_date[4:6] + "-" + txn_date[6:8]
  return txn_date

def data_cleaning(data_set, cityName):
  data_set = data_set.drop([0])
  data_set = data_set.fillna(0)
  data_set['floor_Num'] = data_set['總樓層數'].apply(covert_num)
  data_set['floor_Num'] = data_set['floor_Num'].astype(int)
  data_set['主建物面積'] = data_set['主建物面積'].apply(str)
  data_set['附屬建物面積'] = data_set['附屬建物面積'].apply(str)
  data_set['陽台面積'] = data_set['陽台面積'].apply(str)
  data_set['車位類別'] = data_set['車位類別'].apply(str)
  data_set['移轉層次'] = data_set['移轉層次'].apply(str)
  data_set['總樓層數'] = data_set['總樓層數'].apply(str)
  data_set['主要用途'] = data_set['主要用途'].apply(str)
  data_set['主要建材'] = data_set['主要建材'].apply(str)
  data_set['建築完成年月'] = data_set['建築完成年月'].apply(str)
  data_set['備註'] = data_set['備註'].apply(str)
  data_set['單價元平方公尺'] = data_set['單價元平方公尺'].apply(str)
  data_set['都市土地使用分區'] = data_set['都市土地使用分區'].apply(str)
  data_set['非都市土地使用分區'] = data_set['非都市土地使用分區'].apply(str)
  data_set['非都市土地使用編定'] = data_set['非都市土地使用編定'].apply(str)
  data_set['交易年月日'] = data_set['交易年月日'].apply(convert_western_date)
  data_set.insert(0, 'city', cityName)
  return data_set

df = pd.concat([data_cleaning(df_list[0], "台北市"),
                data_cleaning(df_list[1], "新北市"),
                data_cleaning(df_list[2], "桃園市"),
                data_cleaning(df_list[3], "台中市"),
                data_cleaning(df_list[4], "高雄市")], 
                axis = 0, 
                join="outer",
                ignore_index=True)


dataFrame save to SQLite3 for creating RESTful API

In [ ]:
import sqlite3

#create db
conn = sqlite3.connect('land.db')  
cursor = conn.cursor()
conn.commit()

#replace: Drop the table before inserting new values.
df.to_sql('land_txn_log', conn, if_exists='replace', index=False)
us_df = pd.read_sql("SELECT count(*) FROM land_txn_log;", conn)
print(us_df)

   count(*)
0     49642


Convert to Spark DataFrame is working

In [45]:
#Create PySpark SparkSession
spark = SparkSession.builder \
            .master("local[1]") \
            .appName("SparkMergeDataSet") \
            .getOrCreate()

#Convert Pandas DataFrame to Spark DataFrame
sparkdf_list = [
  spark.createDataFrame(data_cleaning(df_list[0], "台北市")),
  spark.createDataFrame(data_cleaning(df_list[1], "新北市")),
  spark.createDataFrame(data_cleaning(df_list[2], "桃園市")),
  spark.createDataFrame(data_cleaning(df_list[3], "台中市")),
  spark.createDataFrame(data_cleaning(df_list[4], "高雄市"))
]
 
spark_df = reduce(DataFrame.unionAll, sparkdf_list)

print("total data count: ", spark_df.count())

total data count:  49642


- Merge Dataframes by Pyspark
- Filter data by conditions
- Generate JSON files

In [46]:
import json
from pyspark.sql.functions import desc

query_df = spark_df.where('`主要用途` == "住家用"')\
                  .where('`建物型態` like "住宅大樓%"')\
                  .where('floor_num >= 13')\
                  .sort(desc("交易年月日"))

query_df = query_df.drop("floor_num")
pandas_df = query_df.toPandas()
records = pandas_df.values.tolist()

table = {}
for column in records:
  city = column[0]
  district = column[1]
  date = column[8]
  building_state = column[2]
  purpose = column[13]
  floors = column[12]
    
  if not city in table.keys(): 
    table[city] = {}
  if not date in table[city].keys():
    table[city][date] = []
  table[city][date].append({
      "鄉鎮市區": district,
      "建物型態": building_state,
      "主要用途": purpose,
      "總樓層數": floors
  })

result = []

for city, date_table in table.items():
  time_slots = []
  for date, events in date_table.items():
    time_slots.append({
        "date": date,
        "events": events
  })
      
  result.append({
      "city": city,
      "time_slots": time_slots
  })

import io, json
with io.open('result-part1.json', 'w', encoding='utf-8') as f:
  for item in result[:2]:
    f.write(json.dumps(item, ensure_ascii=False, indent=2))

with io.open('result-part2.json', 'w', encoding='utf-8') as f:
  for item in result[2:]:
    f.write(json.dumps(item, ensure_ascii=False))